In [107]:
import yaml, sys
import re

In [146]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [147]:
file_name = "Quiz-1-1-09-between-decimal-and-bin-fractional-numbers"
num_splits = 1

In [148]:
file = open(file_name+".txt", 'r')
gift_text = file.read()
gift_text = gift_text.replace("\{", "%%open_curly%%")
gift_text = gift_text.replace("\}", "%%close_curly%%")
file.close()
#print (gift_text)

In [149]:
questionsrex = "::(.*)::([^{]*){([^}]*)}"
#questionsrex = "::(.*)::\[(.*)\]([^{]*){([^}]*)}"
answersrex = "\t([=~][^\t]*)"

In [150]:
result = re.findall(questionsrex, gift_text)
print(len(result))
print(result[6][1])

12
[html]<p>Convert the hexadecimal number A0.6 to decimal.</p>


In [151]:
qs = []
for res in result:
    qs.append({})
    
    #print (res[1])
    res = [r.replace("<br>", " ") for r in res]
    res = [r.replace("<p>", "\n") for r in res]
    res = [r.replace("</p>", "\n") for r in res]
    res = [r.replace("&nbsp;", "\n") for r in res]
    res = [strip_tags(r) for r in res]
    #print(res[1])
    res = [r.replace("%%open_curly%%", "{") for r in res]
    res = [r.replace("%%close_curly%%", "}") for r in res]
    res = [r.replace("\_", " ") for r in res]
    res = [r.replace("[moodle]", "") for r in res]
    res = [r.replace("[html]", "") for r in res]
    res = [r.replace("[", "\[") for r in res]
    res = [r.replace("]", "\]") for r in res]
    res = [r.replace("\\\\\\hline", "\\\\ \\hline") for r in res]
    res = [r.replace("\\\\", "\\") for r in res]
    def replBackslash(s):
        return s.group(0)[1:]
    #res = [re.sub("\\\\\w", replBackslash, r) for r in res]
    
    
    print(res[2])
    qs[-1]["answers"] = re.findall(answersrex, res[2])
    #res = [r.strip() for r in res]
    
    #print(res[1])
    
    qs[-1]["name"] = res[0]
    qs[-1]["question"] = res[1]
    
    

#
	=%100%26.5625:0#
	=%50%26.5:0.5#

#
	=%100%21.71875:0#
	=%50%21.5:0.5#

#
	=%100%1018.5625:0#
	=%50%1018.5:0.5#

#
	=%100%11010.01:0#
	=%50%11010.5:0.5#

#
	=%100%10101.011:0#
	=%50%10101.5:0.5#

#
	=%100%10010.1001:0#
	=%50%10010.5:0.5#

#
	=%100%160.375:0#
	=%50%160.5:0.5#

#
	=%100%31.4375:0#
	=%50%31.5:0.5#

#
	=%100%3089.1875:0#
	=%50%3089.5:0.5#

#
	=%100%59.09375:0#
	=%50%59.5:0.5#

#
	=%100%111.65625:0#
	=%50%111.5:0.5#

#
	=%100%152.921875:0#
	=%50%152.5:0.5#



In [152]:
print([q['name'] for q in qs])
qs = [q for q in qs if (q["name"].find("fractions of oct") >=0)]
print(qs)

['fractions of bin to dec 1', 'fractions of bin to dec 2', 'fractions of bin to dec 3', 'fractions of dec to bin', 'fractions of dec to bin 2', 'fractions of dec to bin 3', 'fractions of hex  to dec 2', 'fractions of hex to dec', 'fractions of hex to decimal 3', 'fractions of oct to dec 1', 'fractions of oct to dec 3', 'fractions of oct to decimal 2']
[{'name': 'fractions of oct to dec 1', 'answers': ['=%100%59.09375:0#\n', '=%50%59.5:0.5#\n'], 'question': '\nConvert the octal number 73.06 to decimal. \n'}, {'name': 'fractions of oct to dec 3', 'answers': ['=%100%111.65625:0#\n', '=%50%111.5:0.5#\n'], 'question': '\nConvert the octal number 157.52 to decimal \n'}, {'name': 'fractions of oct to decimal 2', 'answers': ['=%100%152.921875:0#\n', '=%50%152.5:0.5#\n'], 'question': '\nConvert the octal number 230.73 to decimal. \n'}]


In [153]:
if num_splits == "all":
    question_list = [[q] for q in qs] 
else: 
    split_size = int(len(qs)/num_splits)
    question_list = [qs[i*split_size: (i+1)*split_size] for i in range(num_splits-1)]
    question_list.append(qs[(num_splits-1)*split_size:])
print(len(qs))
print ([len(q) for q in question_list])

3
[3]


In [154]:
yqs = []
for qs in question_list:
    yqs.append({})
    current_variations = []
    for q in qs :

        yq = {}
        print(q["answers"])
        #if (q["answers"][0][1] == "%"):
        #    yq["typeName"] = "numeric"
        #else:
        yq["typeName"] = "multipleChoice"
        yq["shuffleOptions"] = True
        yq["prompt"] = q["question"].strip()#.replace("\\\\\\hline","\\\\ \\hline").replace("\\\\","\\")#.replace("YAYAYA", "\\")

        yq["options"] = []
        numCorrect = 0
        # HACK: remove this line
        # q["answers"] = q["answers"][:1]
        for answer in q["answers"]:
            a = {}
            answer = answer.strip()
            answertext = answer[1:].strip().split("#")
            a["answer"] = answertext[0].split("%")[-1]
            # in future need to handle this properly for numerical questions
            a["answer"] = a["answer"].split(":")[0]
            percentage = 0
            if(len(answertext[0].split("%")) > 1):
               percentage = int(answertext[0].split("%")[-2])
            print(percentage)
            if(len(answertext)>1):
                a["feedback"] = answer[1:].split("#")[1]
            a["isCorrect"] = True if (answer[0] == "=" or percentage > 99) else False
            if a["isCorrect"] :
                numCorrect += 1 
            yq["options"].append(a)
        print(numCorrect)
        if numCorrect == len(yq["options"]):
           yq["typeName"] = "text"
           yq["answers"] = yq["options"]
           yq["defaultFeedback"] = ""
           del yq["options"]
        if(numCorrect > 1 and yq["typeName"] == "multipleChoice"):
            yq["typeName"] = "checkbox"
        current_variations.append(yq)

    yqs[-1]["variations"] = current_variations
#print (yaml.dump(yqs))


['=%100%59.09375:0#\n', '=%50%59.5:0.5#\n']
100
50
2
['=%100%111.65625:0#\n', '=%50%111.5:0.5#\n']
100
50
2
['=%100%152.921875:0#\n', '=%50%152.5:0.5#\n']
100
50
2


In [155]:
outfile = open(file_name+"-"+str(num_splits)+".yaml", "w")
yaml.dump(yqs, outfile)
outfile.close()

In [52]:
# this is a bit of special code for splitting by type of question, for 1 example

question_list = [[],[],[],[]]

for q in qs:
    #print(q["question"])
    result = re.search("in base ([0-9]*) to base ([0-9]*)", q["question"])
    #result = re.search("in base [0-9]", q["question"])
    #print (result.group(0))
    #print (result.group(1))
    #print (result.group(2))
    if(result.group(1) == "2" and result.group(2) == "8"):
        question_list[0].append(q)
    elif (result.group(1) == "2" and result.group(2) == "16"):
        question_list[1].append(q)
    elif (result.group(1) == "8" and result.group(2) == "2"):
        question_list[2].append(q)
    elif (result.group(1) == "16" and result.group(2) == "2"):
        question_list[3].append(q)
    else: 
        print("error " + result.group(1) + " " + result.group(2))
        
print (question_list)
        

[[], [], [], []]
